In [7]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the results page
url = "https://results.eci.gov.in/PcResultGenJune2024/partywisewinresultState-369.htm"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the relevant table(s)
tables = soup.find_all('table')

# Open a CSV file to write the data
with open('election_results_2024_2.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Iterate through tables and rows
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            data = [column.text.strip() for column in columns]
            writer.writerow(data)

print("Data has been saved to election_results_2024.csv")

Data has been saved to election_results_2024.csv


In [5]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the results page
url = "https://results.eci.gov.in/PcResultGenJune2024/index.htm"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Locate the table containing constituency-wise results
table = soup.find('table')

if table:
    # Open a CSV file to write the data
    with open('constituency_wise_results_2024.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header row
        header = ['Constituency', 'Candidate', 'Party', 'Votes', 'Percentage']
        writer.writerow(header)
        
        # Iterate through the table rows
        for row in table.find_all('tr')[1:]:  # Skip the header row
            columns = row.find_all('td')
            if len(columns) == 5:
                constituency = columns[0].text.strip()
                candidate = columns[1].text.strip()
                party = columns[2].text.strip()
                votes = columns[3].text.strip()
                percentage = columns[4].text.strip()
                data = [constituency, candidate, party, votes, percentage]
                writer.writerow(data)

    print("Constituency-wise data has been saved to constituency_wise_results_2024.csv")
else:
    print("Table not found on the page.")


Constituency-wise data has been saved to constituency_wise_results_2024.csv
